### Write all Politician Information including Votes to S3 based on CongressionalBillsDB- Not For Notebook Jobs

In [2]:
import requests
import re
import ast
import operator
import regex
import os, shutil
import urllib.request
from PIL import Image
import pickle
import time
import sys
import gzip
import os

In [3]:
import boto3
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError
import json
import pandas as pd

#### Read all the legislation from the Congressional Bill DB

In [4]:
def query_Congress_Leg_DB(dynamodb = None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="https://dynamodb.us-east-1.amazonaws.com")
    table = dynamodb.Table('CongressionalBillsDB-iupecu7kgngs3cdnckvjmtrgmm-dev')
    response = table.scan()
    items = response['Items']
    while 'LastEvaluatedKey' in response:
        response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
        items.extend(response['Items'])
    return items

Prior_Voted_Legistlation = query_Congress_Leg_DB()

#### Read all the Politicians stored in DynamoDB

In [5]:
def query_Politicians_DB(dynamodb = None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="https://dynamodb.us-east-1.amazonaws.com")
    table = dynamodb.Table('PoliticiansDB-iupecu7kgngs3cdnckvjmtrgmm-dev')
    response = table.scan()
    items = response['Items']
    while 'LastEvaluatedKey' in response:
        response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
        items.extend(response['Items'])
    return items
Current_DB_Politicians = query_Politicians_DB()

In [6]:
Current_DB_Politicians[1]['NayVotedBills']

['Empty Not Created Yet']

In [7]:
not_in_id_list = []
not_in_name_list = []
not_in_main_id_list = []
not_in_dictionary = {}
for Politician in Current_DB_Politicians:
    if Politician['NayVotedBills'][0] == 'Empty Not Created Yet':
        not_in_main_id_list.append(Politician['id'])
        not_in_id_list.append(Politician['alternateID'])
        not_in_name_list.append(Politician['Name'])
        not_in_dictionary[Politician['Name']] = [Politician['id'], Politician['alternateID']]

In [8]:
not_in_dictionary

{'Eleanor Holmes Norton': ['N000147', 'N000147'],
 'Jack Reed': ['R000122', 'S259'],
 'Margaret Wood Hassan': ['H001076', 'S388'],
 'Celeste Maloy': ['M001228', 'M001228'],
 'J. D. Vance': ['V000137', 'S421'],
 'Michelle Fischbach': ['F000470', 'F000470'],
 'Tim Scott': ['S001184', 'S365'],
 'Ron Wyden': ['W000779', 'S247'],
 'Catherine Cortez Masto': ['C001113', 'S385'],
 'Mazie K. Hirono': ['H001042', 'S361'],
 'Eric Sorensen': ['S001225', 'S001225'],
 'Nathaniel Moran': ['M001224', 'S347'],
 'Lisa Murkowski': ['M001153', 'S288'],
 'Jennifer L. McClellan': ['M001227', 'M001227'],
 'Mark R. Warner': ['W000805', 'S327'],
 'John Cornyn': ['C001056', 'S287'],
 'Raphael G. Warnock': ['W000790', 'S415'],
 'Brian Schatz': ['S001194', 'S353'],
 'Erin Houchin': ['H001093', 'H001093'],
 'James Lankford': ['L000575', 'S378'],
 'Rand Paul': ['P000603', 'S348'],
 'Lindsey Graham': ['G000359', 'S293'],
 'Christopher Murphy': ['M001169', 'S364'],
 'Kevin Cramer': ['C001096', 'S398'],
 'Jon Tester':

#### Write all the politician information with the corresponding legislation information on each voted bill to a JSON file and then save to S3

In [ ]:
for politician in Current_DB_Politicians:

    pol_aye_vote_dict = {}
    for pol_bill in politician['AyeVotedBills']:
        for bill in Prior_Voted_Legistlation:
            if bill['id'] == pol_bill:
                pol_aye_vote_dict[pol_bill] = bill
                pol_aye_vote_dict[pol_bill]['Aye'] = ["Empty for Data Retrieval Purposes"]
                pol_aye_vote_dict[pol_bill]['Nay'] = ["Empty for Data Retrieval Purposes"]
                pol_aye_vote_dict[pol_bill]['Not_Voting'] = ["Empty for Data Retrieval Purposes"]
                break
    politician['AyeVotedBills'] = pol_aye_vote_dict

    pol_nay_vote_dict = {}
    for pol_bill in politician['NayVotedBills']:
        for bill in Prior_Voted_Legistlation:
            if bill['id'] == pol_bill:
                pol_nay_vote_dict[pol_bill] = bill
                pol_nay_vote_dict[pol_bill]['Aye'] = ["Empty for Data Retrieval Purposes"]
                pol_nay_vote_dict[pol_bill]['Nay'] = ["Empty for Data Retrieval Purposes"]
                pol_nay_vote_dict[pol_bill]['Not_Voting'] = ["Empty for Data Retrieval Purposes"]
                break

    politician['NayVotedBills'] = pol_nay_vote_dict

    pol_no_vote_dict = {}
    for pol_bill in politician['NotVotedBills']:
        for bill in Prior_Voted_Legistlation:
            if bill['id'] == pol_bill:
                pol_no_vote_dict[pol_bill] = bill
                pol_no_vote_dict[pol_bill]['Aye'] = ["Empty for Data Retrieval Purposes"]
                pol_no_vote_dict[pol_bill]['Nay'] = ["Empty for Data Retrieval Purposes"]
                pol_no_vote_dict[pol_bill]['Not_Voting'] = ["Empty for Data Retrieval Purposes"]
                break

    politician['NotVotedBills'] = pol_no_vote_dict

    # Specify the filename
    filename = "PoliticianJSONFolder/" + politician['id'] + '.json'
    # Convert Decimal to string during serialization

    # Open the file in write mode ('w') and write the dictionary
    with open(filename, 'w') as file:
        json.dump(politician, file, default = str, indent=4)  # Use 'indent' for pretty printing

    bucket_name = "moderateapps3bucket01959-dev"
    file_name_in_s3 = "public/" + filename

    # json_str = json.dumps(politician, default=str)
    # print(type(json_str))
    # json_data = json.loads(json_str)

    s3 = boto3.client('s3')
    with open(filename, "rb") as f:
        bucket = "moderateapps3bucket01959-dev"
        save_string = "public/" + filename
        s3.upload_fileobj(f, bucket, save_string)

#### Delete All the JSON files in the Folder

In [ ]:
# Specify the directory you want to clear
folder_path = 'PoliticianJSONFolder/'

# Loop through all files in the directory
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        # Check if it is a file and not a directory
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path) # This will delete the file
        # Uncomment below if you also want to remove subdirectories
        # elif os.path.isdir(file_path):
        #     shutil.rmtree(file_path) # Be careful with this, it deletes directories
    except Exception as e:
        print(f'Failed to delete {file_path}. Reason: {e}')


In [ ]:
id_list = []

name_list = []

name_id_dict = {}

In [ ]:
import json

# Open the JSON file for reading
with open('data.json', 'r') as file:
    # Parse the JSON file and convert it into a Python dictionary
    data = json.load(file)

In [ ]:


# for vote in data['votes']['No']:
#     id_list.append(vote['id'])
#     name_list.append(vote['display_name'])
#     name_id_dict[vote['display_name']] = vote['id']
for vote in data['votes']['Aye']:
    id_list.append(vote['id'])
    name_list.append(vote['display_name'])
    name_id_dict[vote['display_name']] = vote['id']
for vote in data['votes']['Not Voting']:
    id_list.append(vote['id'])
    name_list.append(vote['display_name'])
    name_id_dict[vote['display_name']] = vote['id']

In [ ]:
name_id_dict